<a href="https://colab.research.google.com/github/Guiils/analise_de_dados/blob/main/Tratamento_dados_atendimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gcsfs

In [ ]:
pip install pandera

In [ ]:
import pandas as pd
import numpy as np
import pandera as pa
import os
from google.cloud import storage

In [ ]:
serviceAccount = '/content/advance-airline-377819-1bf613aac07c.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

##EXTRAÇÃO

In [ ]:
path = 'gs://ad1-bucket/brutos/dados_atendimento - atendimentos.csv'

In [ ]:
df = pd.read_csv(path)

In [ ]:
df

,Call Id,Data,Atendente,Área,Respondido - Resolvido (S/N),Velocidade de Resposta,Duracao Chamada,Duracao com atendente em Segundos,Índice de Satisfacao
0,ID0001,43912,João D.,Vendas,Sim - S,109,00:02:23,143,2
1,ID0002,43907,Marcelo F.,RECLAMAÇÕES,Sim - N,70,00:04:02,242,3
2,ID0003,43908,Leandro C.,Vendas,Sim - S,10,00:02:11,131,3
3,ID0004,43899,Leandro C.,Vendas,Sim - S,53,00:00:37,37,2
4,ID0007,43891,Marina Z.,Cancelametos,Sim - S,24,00:03:40,220,2
...,...,...,...,...,...,...,...,...,...
1004,ID1764,43896,Ana S.,Vendas,Sim - S,92,00:06:00,360,4
1005,ID1765,43895,Ana S.,Vendas,Sim - S,95,00:05:00,300,3
1006,ID1766,43910,Gustavo M.,Vendas,Sim - S,49,00:01:58,118,5
1007,ID1770,43894,Ana S.,Cancelametos,Sim - S,78,00:01:02,62,4


##TRATAMENTO

In [ ]:
df.loc[df['Área'] == 'Cancelametos', 'Área'] = 'Cancelamentos'
df.loc[df['Área'] == 'RECLAMAÇÕES', 'Área'] = 'Reclamações'

In [ ]:
df['Resolvido'] = df.loc[df['Respondido - Resolvido (S/N)']=='Sim - S','Resolvido'] = 'S'
df.loc[df['Respondido - Resolvido (S/N)']=='Sim - N','Resolvido'] = 'N'

In [ ]:
df.rename(columns={'Respondido - Resolvido (S/N)': 'Respondido'}, inplace=True)

In [ ]:
df.loc[df['Respondido']=='Sim - S','Respondido'] = 'Sim'
df.loc[df['Respondido']=='Sim - N','Respondido'] = 'Sim'
df.loc[df['Respondido']=='Não - N','Respondido'] = 'Não'

In [ ]:
# Converte a coluna de data serial em um objeto DateTime
df['Data'] = pd.to_datetime(df['Data'], unit='D', origin='1899-12-30')

# Formata a data no estilo 'dia/mês/ano'
df['Data'] = df['Data'].dt.strftime('%m/%d/%Y')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Call Id                            1009 non-null   object
 1   Data                               1009 non-null   object
 2   Atendente                          1009 non-null   object
 3   Área                               1009 non-null   object
 4   Respondido                         1009 non-null   object
 5   Velocidade de Resposta             1009 non-null   int64 
 6   Duracao Chamada                    1009 non-null   object
 7   Duracao com atendente em Segundos  1009 non-null   int64 
 8   Índice de Satisfacao               1009 non-null   int64 
 9   Resolvido                          1009 non-null   object
dtypes: int64(3), object(7)
memory usage: 79.0+ KB


In [ ]:
df

,Call Id,Data,Atendente,Área,Respondido,Velocidade de Resposta,Duracao Chamada,Duracao com atendente em Segundos,Índice de Satisfacao,Resolvido
0,ID0001,03/22/2020,João D.,Vendas,Sim,109,00:02:23,143,2,S
1,ID0002,03/17/2020,Marcelo F.,Reclamações,Sim,70,00:04:02,242,3,N
2,ID0003,03/18/2020,Leandro C.,Vendas,Sim,10,00:02:11,131,3,S
3,ID0004,03/09/2020,Leandro C.,Vendas,Sim,53,00:00:37,37,2,S
4,ID0007,03/01/2020,Marina Z.,Cancelamentos,Sim,24,00:03:40,220,2,S
...,...,...,...,...,...,...,...,...,...,...
1004,ID1764,03/06/2020,Ana S.,Vendas,Sim,92,00:06:00,360,4,S
1005,ID1765,03/05/2020,Ana S.,Vendas,Sim,95,00:05:00,300,3,S
1006,ID1766,03/20/2020,Gustavo M.,Vendas,Sim,49,00:01:58,118,5,S
1007,ID1770,03/04/2020,Ana S.,Cancelamentos,Sim,78,00:01:02,62,4,S


##LOAD (CARREGAMENTO)

In [ ]:
df.to_csv('gs://ad1-bucket/tratados/dados_atendimento - atendimentos_tratado.csv', index = False)

Link para visualização do Dashboard montado em power bi:

https://app.powerbi.com/view?r=eyJrIjoiNDhjMzk3MDktYjJhMy00Njg1LWIzNjgtOGU4NjdkZGZhZjA3IiwidCI6IjM1ZWFlNjdjLTY4NDctNDA5Mi1hMDk5LThkMGVhYTM1Nzc3ZiJ9&pageName=ReportSection